In [ ]:
import copy
import glob
import japanize_matplotlib
import math
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import pandas as pd
import pytest
import random
import sys
from sklearn import linear_model
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as sp
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from unittest.mock import MagicMock

In [ ]:
# 平均絶対パーセント誤差 (MAPE)(Mean Absolute Percent Error (MAPE))を返す関数
# 引数として長さの同じ二つのリストをとる
# 引数l1: 実測値のリスト
# 引数l2: 予測値のリスト
# 単位：％

def returnMapeScore(l1, l2):
    return_num = 0
    if(len(l1) != len(l2)):
        print("引数のリストの長さが異なります", end=", ")
        return -1
    for i in range(len(l1)):
        l1_num = l1[i]
        l2_num = l2[i]

        return_num += abs((l1_num - l2_num)/l1_num)

    return_num /= len(l1)
    return_num *= 100
    return return_num


# 使用例：returnMapeScore([1,2,3,4], [4,3,2,1])
type(returnMapeScore([1, 2, 3, 4], [4, 3, 2, 1]))


def test_returnMapeScore():
    l1 = [1, 2, 3, 4]
    l2 = [4, 3, 2, 1]
    ansByFunc = returnMapeScore(l1, l2)
    ansByHand = (abs(1-4)/1 + abs(2-3)/2 + abs(3-2)/3 + abs(4-1)/4)/4 * 100
    # 多少の誤差を許容する
    ansByFunc = int(ansByFunc * 100) / 100
    ansByHand = int(ansByHand * 100) / 100

    assert ansByFunc == ansByHand

In [ ]:
# ベンチマークを指定して存在するファイル名のものを返す
def returnExistingFileNames(benchmarkNames=[], classes=[], processes=[], csvDirPath="./csv_files"):
    candidateFileNames = {}
    returnDict = {}
    for benchmarkName in benchmarkNames:
        for benchmarkClass in classes:
            for process in processes:
                candidateFileNames[f"pprof_{benchmarkName}{benchmarkClass}{process}.csv"] = {
                    "benchmarkName": benchmarkName, "benchmarkClass": benchmarkClass, "process": process}
    for candidateFileName in candidateFileNames.keys():
        filePath = os.path.join(csvDirPath, candidateFileName)
        if(os.path.exists(filePath) and os.stat(filePath).st_size != 0):
            returnDict[candidateFileName] = candidateFileNames[candidateFileName]
    return(returnDict)


def test_returnExistingFileNames():
    benchmarkNames = ["test"]
    classes = ["A", "B", "C", "D"]
    processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    csvDirPath = "../csv_files/"
    returnedList = returnExistingFileNames(
        benchmarkNames=benchmarkNames, classes=classes, processes=processes, csvDirPath=csvDirPath)
#     print(returnedList)
    assert returnedList["pprof_testA128.csv"] == {
        "benchmarkName": "test", "benchmarkClass": "A", "process": 128}
    assert returnedList["pprof_testB256.csv"] == {
        "benchmarkName": "test", "benchmarkClass": "B", "process": 256}

In [ ]:
# ベンチマーク名・プロセス数・ベンチマーククラスをリストで渡して、実在するデータが集計されたDFを返す
def returnCollectedExistingData(benchmarkNames=[], classes=[], processes=[], csvDirPath="./csv_files"):
    fileNames = returnExistingFileNames(
        benchmarkNames=benchmarkNames, classes=classes, processes=processes, csvDirPath=csvDirPath)
    csvDataList = []
    for fileName in fileNames.keys():
        rawDatum = pd.read_csv(f"{csvDirPath}{fileName}")
        rawDatum["benchmarkName"] = fileNames[fileName]["benchmarkName"]
        rawDatum["benchmarkClass"] = fileNames[fileName]["benchmarkClass"]
        rawDatum["process"] = fileNames[fileName]["process"]
        csvDataList.append(rawDatum)
    returnDF = pd.concat(csvDataList, axis=0)
    returnDF = returnDF.rename(
        columns={'Name': 'functionName', '#Call': 'functionCallNum'})
    return(returnDF)


def test_returnCollectedExistingData():
    benchmarkNames = ["test"]
    classes = ["A", "B", "C", "D"]
    processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    csvDirPath = "../csv_files/"
    returnedData = returnCollectedExistingData(
        benchmarkNames=benchmarkNames, classes=classes, processes=processes, csvDirPath=csvDirPath)

    case01 = {"benchmarkName": "test", "benchmarkClass": "A", "process": 128,
              "functionCalls": {"function00": 99, "function01": 77, "function02": 555}}
    case02 = {"benchmarkName": "test", "benchmarkClass": "B", "process": 256,
              "functionCalls": {"function00": 5, "function01": 70, "function02": 900}}

    for case in [case01, case02]:
        benchmarkName = case["benchmarkName"]
        benchmarkClass = case["benchmarkClass"]
        process = case["process"]
        for functionName in case["functionCalls"]:
            functionCallNum = case["functionCalls"][functionName]
            targetData = returnedData[(returnedData['benchmarkName'] == benchmarkName) & (returnedData['benchmarkClass'] == benchmarkClass) & (
                returnedData["process"] == process) & (returnedData["functionName"] == functionName)]
            columns = targetData.columns.tolist()
            functionCallNumIndex = columns.index("functionCallNum")
            assert targetData.iloc[0, functionCallNumIndex] == functionCallNum

In [ ]:
# モデルの共通部分となるクラス
# すべての引数はただのリスト。クラスの初期化時に""np.reshape()""を実行する
class ModelBase:
    def __init__(self, trainX, trainY, targetX=[], targetY=[], benchmarkName="benchmarkName", functionName="functionName"):
        self.benchmarkName = benchmarkName
        self.functionName = functionName

        self.trainX = np.reshape(trainX, (-1, 1))
        self.trainY = np.reshape(trainY, (-1, 1))
        self.targetX = np.reshape(targetX, (-1, 1))
        self.targetY = np.reshape(targetY, (-1, 1))

    def returnTargetX():
        return(self.targetX)

    def returnTargetY():
        return(self.targetY)

    def returnTrainX():
        return(self.trainX)

    def returnTrainY():
        return(self.trainY)

# # このクラスを継承したモデルは、いずれも次のように使用する
# _modelLin = ModelLin(trainX=trainX, trainY=trainY, targetX=targetX, targetY=targetY)
# _modelLin.calcLr()
# plotY = _modelLin.predict(plotX)

In [ ]:
# 分岐モデル

class ModelBranch(ModelBase):
    def calcLr(self):
        self.t = np.ndarray.argmax(self.trainY)
        self.tNum = self.trainX[self.t]
        if (self.t == 0 or self.t == len(self.trainY) - 1):
            self.lr1 = LinearRegression()
            self.lr1.fit(self.trainX, self.trainY)
            self.lr2 = LinearRegression()
            self.lr2.fit(self.trainX, self.trainY)
        else:
            self.trainX1 = self.trainX[:self.t]
            self.trainX2 = self.trainX[self.t:]
            self.trainY1 = self.trainY[:self.t]
            self.trainY2 = self.trainY[self.t:]
            self.lr1 = LinearRegression()
            self.lr1.fit(self.trainX1, self.trainY1)
            self.lr2 = LinearRegression()
            self.lr2.fit(self.trainX2, self.trainY2)

    def predict(self, num):
        num = np.reshape(num, (-1, 1))
        numT = np.ndarray.argmax(num)
        numTMax = num[numT]
        k = np.abs(np.asarray(num) - self.tNum).argmin()
        if(len(num) == 1 and numTMax >= self.tNum):
            predicted = self.lr2.predict(num)
            return(predicted)
        elif (numTMax < self.trainX[self.t] or k == 0):
            predicted = self.lr1.predict(num)
            return(predicted)
        else:
            num1 = num[:k]
            num2 = num[k:]
            predicted1 = self.lr1.predict(num1)
            predicted2 = self.lr2.predict(num2)
            predicted = np.concatenate([predicted1, predicted2])
            return(predicted)

    def ModelName(self):
        return("ModelBranch")

# 線形飽和モデル
# テスト用モデル式1：
# y = 2x + 3 (x < 10)
#     23     (x >= 10)
# テスト用モデル式2：
# y = 2x + 3


def test_ModelBranch():
    # X軸の値
    plotXForBranch = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    # Y軸の値
    plotYForBranch = [5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 23, 23, 23, 23]
#     plt.figure()
#     plt.plot(plotXForBranch, plotYForBranch, label="y=2*+3(x<10), y=23(x>=10)")
    # モデルの構築
    _modelBranch = ModelBranch(
        trainX=plotXForBranch, trainY=plotYForBranch, targetX=[], targetY=[])
    _modelBranch.calcLr()
    predictedYForBranch = _modelBranch.predict(plotXForBranch)
#     plt.plot(plotXForBranch, predictedYForBranch, label="線形飽和モデルによるモデル式")
#     plt.legend()
    mapeScore = returnMapeScore(plotYForBranch, predictedYForBranch)
    assert mapeScore < 1

In [ ]:
# 反比例モデル

def ipFunc(x):
    return 1/x


class ModelIp(ModelBase):

    def calcLr(self):
        self.transformerIp = sp.FunctionTransformer(
            func=ipFunc, inverse_func=ipFunc)
        trainXIp = self.transformerIp.transform(self.trainX)
        self.lr = LinearRegression()
        self.lr.fit(trainXIp, self.trainY)

    def predict(self, num):
        num = np.reshape(num, (-1, 1))
        numConverted = self.transformerIp.transform(num)
        predicted = self.lr.predict(numConverted)
        return(predicted)

    def return_coef_(self):
        return self.lr.coef_

    def return_intercept_(self):
        return self.lr.intercept_

    def ModelName(self):
        return("ModelIp")

# 反比例モデル
# テスト用モデル式：
# y = 2/x + 3


def test_ModelIp():
    # X軸の連続値
    plotX = np.linspace(0.5, 270, 500)
#     plt.figure()
    plotY = 2/plotX + 3
#     plt.plot(plotX, plotY, label="y = 2/x + 3")
    # モデルの構築
    _modelIp = ModelIp(trainX=plotX, trainY=plotY, targetX=[], targetY=[])
    _modelIp.calcLr()
    predictedY = _modelIp.predict(plotX)
#     plt.plot(plotX, predictedY, label="反比例モデルによるモデル式")
#     plt.legend()
    mapeScore = returnMapeScore(plotY, predictedY)
    assert mapeScore < 1

In [ ]:
# 線形モデル

class ModelLin(ModelBase):

    def calcLr(self):
        self.lr = LinearRegression()
        self.lr.fit(self.trainX, self.trainY)

    def predict(self, num):
        num = np.reshape(num, (-1, 1))
        predicted = self.lr.predict(num)
        return(predicted)

    def return_coef_(self):
        return self.lr.coef_

    def return_intercept_(self):
        return self.lr.intercept_

    def ModelName(self):
        return("ModelLin")

# 線形モデル
# テスト用モデル式：
# y = 2x + 3


def test_ModelLin():
    # X軸の連続値
    plotX = np.linspace(0.5, 270, 500)
#     plt.figure()
    plotY = 2 * plotX + 3
#     plt.plot(plotX, plotY, label="y = 2 * x + 3")
    # モデルの構築
    _modelLin = ModelLin(trainX=plotX, trainY=plotY, targetX=[], targetY=[])
    _modelLin.calcLr()
    predictedY = _modelLin.predict(plotX)
#     plt.plot(plotX, predictedY, label="線形モデルによるモデル式")
#     plt.legend()
    mapeScore = returnMapeScore(plotY, predictedY)
    assert mapeScore < 1

In [ ]:
# 対数モデル


def inverterLog10Func(x):
    return 10**x


class ModelLog10(ModelBase):

    def calcLr(self):
        self.transformerLog10 = sp.FunctionTransformer(
            func=np.log10, inverse_func=inverterLog10Func)
        trainXLog10 = self.transformerLog10.transform(self.trainX)
        self.lr = LinearRegression()
        self.lr.fit(trainXLog10, self.trainY)

    def predict(self, num):
        num = np.reshape(num, (-1, 1))
        numConverted = self.transformerLog10.transform(num)
        predicted = self.lr.predict(numConverted)
        return(predicted)

    def return_coef_(self):
        return self.lr.coef_

    def return_intercept_(self):
        return self.lr.intercept_

    def ModelName(self):
        return("ModelLog")

# 対数モデル
# テスト用モデル式：
# y = 2 * log_{10}{x} + 3


def test_ModelLog10():
    # X軸の連続値
    plotX = np.linspace(0.5, 270, 500)
#     plt.figure()
    plotY = 2 * np.log10(plotX) + 3
#     plt.plot(plotX, plotY, label="y = 2 * log_{10}{x} + 3")
    # モデルの構築
    _modelLog10 = ModelLog10(
        trainX=plotX, trainY=plotY, targetX=[], targetY=[])
    _modelLog10.calcLr()
    predictedY = _modelLog10.predict(plotX)
#     plt.plot(plotX, predictedY, label="対数モデルによるモデル式")
#     plt.legend()
    mapeScore = returnMapeScore(plotY, predictedY)
    assert mapeScore < 1

In [ ]:
# 引数として渡されたDFに
# functionName, functionCallNum, benchmarkName, benchmarkClass, process
# のカラムがあるかを確認する関数
# あればTrue、なければFalseを返す
def checkRawDFColumns(DF):
    columns = DF.columns.tolist()
    columnNames = ["functionName", "functionCallNum",
                   "benchmarkName", "benchmarkClass", "process"]
    for columnName in columnNames:
        if((columnName in columns) == False):
            print(columnName)
            return False
    return True


def test_checkRawDFColumns():
    # Trueケース
    True01DF = pd.DataFrame(
        [["functionName", -1, "benchmarkName", "Z", -1]],
        columns=["functionName", "functionCallNum",
                 "benchmarkName", "benchmarkClass", "process"]
    )
    True02DF = pd.DataFrame(
        [["functionName", -1, "benchmarkName", "Z", -1, "addedData0"]],
        columns=["functionName", "functionCallNum", "benchmarkName",
                 "benchmarkClass", "process", "addedData0"]
    )

    # Falseケース
    False01DF = pd.DataFrame(
        [["functionName", -1, "benchmarkName", "Z"]],
        columns=["functionName", "functionCallNum",
                 "benchmarkName", "benchmarkClass"]
    )
    False02DF = pd.DataFrame(
        [[-1, "benchmarkName", "Z", -1]],
        columns=["functionCallNum", "benchmarkName",
                 "benchmarkClass", "process"]
    )

    assert True == checkRawDFColumns(True01DF)
    assert True == checkRawDFColumns(True02DF)
    assert False == checkRawDFColumns(False01DF)
    assert False == checkRawDFColumns(False02DF)

In [ ]:
# 実験結果を集計するためのデータフレームのカラムの名称のリストを返す関数
def returnNumOfColumns(dataType=False):
    returnList = []
    returnDict = {}
    # ベンチマーク名
    returnList.append("benchmarkName")
    returnDict["benchmarkName"] = str
    # 関数名
    returnList.append("functionName")
    returnDict["functionName"] = str
    # 使用データ(説明変数のリスト)
    returnList.append("usedDataX")
    returnDict["usedDataX"] = object
    # 使用データ(目的変数のリスト)
    returnList.append("usedDataY")
    returnDict["usedDataY"] = object
    # 使用データ数
    returnList.append("numOfData")
    returnDict["numOfData"] = "int16"
    # 固定したもの("Process" or "Class")
    returnList.append("ProcessOrClass")
    returnDict["ProcessOrClass"] = str
    # 固定したもの(プロセス数(数値)or問題サイズ(文字列))
    returnList.append("fixed")
    returnDict["fixed"] = object
    # 予測対象プロセス数
    returnList.append("targetProcess")
    returnDict["targetProcess"] = "int16"
    # 予測対象問題サイズ
    returnList.append("targetProblemSize")
    returnDict["targetProblemSize"] = str
    # 予測対象関数コール回数
    returnList.append("targetNumOfFunctionCall")
    returnDict["targetNumOfFunctionCall"] = "float32"
    # 線形モデルのオブジェクト
    returnList.append("objectLinModel")
    returnDict["objectLinModel"] = object
    # 線形モデルのMAPE
    returnList.append("MAPEOfLinModel")
    returnDict["MAPEOfLinModel"] = "float32"
    # 反比例モデルのオブジェクト
    returnList.append("objectIpModel")
    returnDict["objectIpModel"] = object
    # 反比例モデルのMAPE
    returnList.append("MAPEOfIpModel")
    returnDict["MAPEOfIpModel"] = "float32"
    # 対数モデルのオブジェクト
    returnList.append("objectLogModel")
    returnDict["objectLogModel"] = object
    # 対数モデルのMAPE
    returnList.append("MAPEOfLogModel")
    returnDict["MAPEOfLogModel"] = "float32"
    # 線形飽和モデルのオブジェクト
    returnList.append("objectBranchModel")
    returnDict["objectBranchModel"] = object
    # 線形飽和モデルのMAPE
    returnList.append("MAPEOfBranchModel")
    returnDict["MAPEOfBranchModel"] = "float32"
    # 説明変数に対するMAPEが最小のモデル名
    returnList.append("objectBestModelName")
    returnDict["objectBestModelName"] = object
    # 説明変数に対するMAPEが最小のモデルを用いて予測対象の関数コール回数を予測した時の絶対相対誤差率[%]
    returnList.append("MAPEOfBestModel")
    returnDict["MAPEOfBestModel"] = "float32"
    if(dataType == True):
        return(returnDict)
    else:
        return(returnList)

# 使用例
# columnNames = return_numOfColumns()
# df_sample = pd.DataFrame(columns=columnNames)
# df_sample


def test_returnNumOfColumns():
    returnedList = returnNumOfColumns()
    returnedDict = returnNumOfColumns(dataType=True)
    # カラム名と辞書のキーが一致しているかを確認
    for key in returnedDict.keys():
        assert (key in returnedList)
    # カラム名を返す場合にリスト長が想定通りかどうかを確認
    assert len(returnedList) == 20
    # カラム名を返す場合に辞書のキー数が想定通りかどうかを確認
    assert len(returnedDict.keys()) == 20

In [ ]:
def returnSpecificDataFromCSV(benchmarkName="cg", functionName=".TAU_application", process="1", benchmarkClass="A", csvDirPath="./csv_files"):
    fileName = f"pprof_{benchmarkName}{benchmarkClass}{process}.csv"
    filePath = f"{csvDirPath}/{fileName}"
    rawCSVData = pd.read_csv(filePath)
    rawCSVDataPerFunction = rawCSVData[(
        rawCSVData["Name"] == functionName)].set_index("Name")
    returnData = rawCSVDataPerFunction.at[functionName, "#Call"]
    return(returnData)


def test_returnSpecificDataFromCSV():
    case01 = {"benchmarkName": "test", "benchmarkClass": "A",
              "process": 128, "functionName": "function00", "functionCallNum": 99}
    case02 = {"benchmarkName": "test", "benchmarkClass": "B",
              "process": 256, "functionName": "function02", "functionCallNum": 900}
    for case in [case01, case02]:
        benchmarkName = case["benchmarkName"]
        benchmarkClass = case["benchmarkClass"]
        process = case["process"]
        functionName = case["functionName"]
        functionCallNum = case["functionCallNum"]
        assert functionCallNum == returnSpecificDataFromCSV(
            benchmarkName=benchmarkName, functionName=functionName, process=process, benchmarkClass=benchmarkClass, csvDirPath="../csv_files")

In [ ]:
def convertStrToInt_problemSizeInNPB(Alphabet: str):
    if(Alphabet == "A"):
        return (1)
    elif(Alphabet == "B"):
        return (4)
    elif(Alphabet == "C"):
        return (16)
    elif(Alphabet == "D"):
        return (256)
    else:
        return False

def test_convertStrToInt_problemSizeInNPB():
    case00 = {"input":"A", "output":1}
    case01 = {"input":"Z", "output":False}
    
    for case in [case00 , case01]:
        output = convertStrToInt_problemSizeInNPB(case["input"])
        assert output == case["output"] 


def convertIntToStr_problemSizeInNPB(number):
    number = int(number)
    if(number == 1):
        return("A")
    elif(number == 4):
        return("B")
    elif(number == 16):
        return("C")
    elif(number == 256):
        return("D")
    else:
        return False
    
def test_convertIntToStr_problemSizeInNPB():
    case00 = {"input":1, "output":"A"}
    case01 = {"input":-1, "output":False}
    
    for case in [case00, case01]:
        output = convertIntToStr_problemSizeInNPB(case["input"])
        assert output == case["output"]


def convertBenchmarkClasses_problemSizeInNPB(inputList=["A", "B", "C", "D"]):
    ReturnList = []
    for content in inputList:
        ReturnList.append(convertStrToInt_problemSizeInNPB(content))
    return(ReturnList)

def test_convertBenchmarkClasses_problemSizeInNPB():
    case00 = {"input":["A", "B", "C", "D"], "output":[1,4,16,256]}
    case01 = {"input":["D", "A"], "output":[256, 1]}
    case02 = {"input":["A", "X", "Y", "Z"], "output":[1, False, False, False]}
    
    for case in [case00, case01, case02]:
        returnedList = convertBenchmarkClasses_problemSizeInNPB(inputList=case["input"])
        assert returnedList == case["output"]
        

# return_numOfColumns()でのカラム名としてのモデル名、モデルのメソッドModelName()が返すモデル名を相互的なキー・バリューとした辞書を返す関数
def returnDictModelNames():
    returnDict = {}
    # カラム名をキー・モデルが返すモデル名をバリュー
    returnDict["objectLinModel"] = "ModelLin"
    returnDict["objectIpModel"] = "ModelIp"
    returnDict["objectLogModel"] = "ModelLog"
    returnDict["objectBranchModel"] = "ModelBranch"
    # モデルが返すモデル名をキー・カラム名をバリュー
    returnDict["ModelLin"] = "objectLinModel"
    returnDict["ModelIp"] = "objectIpModel"
    returnDict["ModelLog"] = "objectLogModel"
    returnDict["ModelBranch"] = "objectBranchModel"

    return(returnDict)

In [ ]:
# 結果を集計するためのDFに挿入するSeriesを作成する関数
def returnSeriesOfData(benchmarkName="benhmarkName", functionName="functionName", rawX=[1, 2, 3], rawY=[1, 2, 3], fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files"):

    dataSeries = pd.Series(index=returnNumOfColumns(), dtype=object)
    dataSeries["benchmarkName"] = benchmarkName
    dataSeries["functionName"] = functionName
    dataSeries["usedDataX"] = rawX
    dataSeries["usedDataY"] = rawY
    dataSeries["numOfData"] = len(rawX)
    dataSeries["ProcessOrClass"] = fixProcessOrClass
    dataSeries["fixed"] = fixed
    dataSeries["targetProcess"] = targetProcess
    dataSeries["targetBenchmarkClass"] = targetBenchmarkClass

    dataSeries["targetNumOfFunctionCall"] = returnSpecificDataFromCSV(
        benchmarkName=benchmarkName, functionName=functionName, process=targetProcess, benchmarkClass=targetBenchmarkClass, csvDirPath=csvDirPath)

#     # MAPE の算出には returnMapeScore()を用いる
#     # returnMapeScore()の返り値の単位は％

#     # 線形モデル
    modelLin = ModelLin(trainX=rawX, trainY=rawY)
    modelLin.calcLr()
    predictedY = modelLin.predict(rawX)
    dataSeries["objectLinModel"] = modelLin
    dataSeries["MAPEOfLinModel"] = returnMapeScore(predictedY, rawY)
    # 反比例モデル
    modelIp = ModelIp(trainX=rawX, trainY=rawY)
    modelIp.calcLr()
    predictedY = modelIp.predict(rawX)
    dataSeries["objectIpModel"] = modelIp
    dataSeries["MAPEOfIpModel"] = returnMapeScore(predictedY, rawY)
    # 対数モデル
    modelLog = ModelLog10(trainX=rawX, trainY=rawY)
    modelLog.calcLr()
    predictedY = modelLog.predict(rawX)
    dataSeries["objectLogModel"] = modelLog
    dataSeries["MAPEOfLogModel"] = returnMapeScore(predictedY, rawY)
    # 分岐モデル
    modelBranch = ModelBranch(trainX=rawX, trainY=rawY)
    modelBranch.calcLr()
    predictedY = modelBranch.predict(rawX)
    dataSeries["objectBranchModel"] = modelBranch
    dataSeries["MAPEOfBranchModel"] = returnMapeScore(predictedY, rawY)
    # 最適なモデルのモデルのモデル名・MAPE値の算出
    listToCalcBestModel = {}
    listToCalcBestModel[dataSeries["objectLinModel"].ModelName(
    )] = dataSeries["MAPEOfLinModel"]
    listToCalcBestModel[dataSeries["objectIpModel"].ModelName(
    )] = dataSeries["MAPEOfIpModel"]
    listToCalcBestModel[dataSeries["objectLogModel"].ModelName(
    )] = dataSeries["MAPEOfLogModel"]
    listToCalcBestModel[dataSeries["objectBranchModel"].ModelName(
    )] = dataSeries["MAPEOfBranchModel"]
    minMAPE = min(listToCalcBestModel.values())
    dataSeries["MAPEOfBestModel"] = minMAPE
    dataSeries["objectBestModelName"] = [
        k for k, v in listToCalcBestModel.items() if v == minMAPE][0]
    # relativeErrorOfBestModelへのデータ格納処理
    # これには、学習したモデルから対象となる関数コール回数を予測し、予測値と実測値の相対誤差を入れる
    dictOfModelNames = returnDictModelNames()
    bestModelName = dataSeries["objectBestModelName"]
    bestModelColumnName = dictOfModelNames[bestModelName]
    bestModelObject = dataSeries[bestModelColumnName]
    if(fixProcessOrClass == "Class"):
        targetX = targetProcess
    else:
        targetX = convertStrToInt_problemSizeInNPB(targetProblemSize)
#     predictedTargetData = bestModelObject.predict(targetX).tolist()[0]
#     dataSeries["predictedTargetNumOfFunctionCall"] = predictedTargetData
#     dataSeries["relativeErrorOfBestModel"] = returnRelativeErrorRate(
#         real_data=dataSeries["targetNumOfFunctionCall"], predict_data=predictedTargetData)

    return(dataSeries)


@pytest.fixture()
def test_generateCSVFilesForReturnSeriesOfData():
    filePath = "/tmp/pprof_testD256.csv"
    functionName = "testFunctionName"
    with open(filePath, 'w') as f:
        f.write("Name,#Call\n")
        # 本来は各モデルごとに最適な関数コール回数とするべきだが、できないので-1を返すようにした
        f.write(f"{functionName},-1\n")


def test_returnSeriesOfData(test_generateCSVFilesForReturnSeriesOfData):
    # 共通部分の設定
    benchmarkName = "test"
    functionName = "testFunctionName"
    targetProcess = 256
    targetBenchmarkClass = "D"
    fixProcessOrClass = "Class"
    fix = targetBenchmarkClass

    csvDirPathForTest = "/tmp"

    explanatoryVariableX = np.array([1, 2, 4, 8, 16, 32, 64, 128, 256])
    responseVariableY = [-1, -1, -1, -1, -1, -1, -1, -1, -1]

    # 分岐モデルが最適となる場合
    # 目的変数を分岐モデルが最適となるように設定する
    responseVariableY = [48, 52, 60, 76, 108, 172, 300, 300, 300]
    branchSeries = returnSeriesOfData(benchmarkName=benchmarkName, functionName=f"{functionName}", rawX=explanatoryVariableX, rawY=responseVariableY,
                                      fixProcessOrClass=fixProcessOrClass, fixed=targetBenchmarkClass, targetProcess=targetProcess, targetBenchmarkClass=targetBenchmarkClass, targetFunctionCallNum=responseVariableY[-1], csvDirPath=csvDirPathForTest)
    # 最適なモデルが分岐モデルであることを確認
    assert branchSeries["objectBestModelName"] == "ModelBranch"
    # 説明変数に対するMAPEが最小のモデルを用いて予測対象の関数コール回数を予測した時の絶対相対誤差率が非常に小さいことを確認

    # 反比例モデルが最適となる場合
    # 目的変数を反比例モデルが最適となるように設定する
    responseVariableY = 2/explanatoryVariableX + 3
    ipSeries = returnSeriesOfData(benchmarkName=benchmarkName, functionName=f"{functionName}", rawX=explanatoryVariableX, rawY=responseVariableY,
                                  fixProcessOrClass=fixProcessOrClass, fixed=targetBenchmarkClass, targetProcess=targetProcess, targetBenchmarkClass=targetBenchmarkClass, csvDirPath=csvDirPathForTest)
    # 最適なモデルが反比例モデルであることを確認
    assert ipSeries["objectBestModelName"] == "ModelIp"
    # 説明変数に対するMAPEが最小のモデルを用いて予測対象の関数コール回数を予測した時の絶対相対誤差率が非常に小さいことを確認

    # 線形モデルが最適となる場合
    # 目的変数を線形モデルが最適となるように設定する
    responseVariableY = 2 * explanatoryVariableX + 3
    linSeries = returnSeriesOfData(benchmarkName=benchmarkName, functionName=f"{functionName}", rawX=explanatoryVariableX, rawY=responseVariableY,
                                   fixProcessOrClass=fixProcessOrClass, fixed=targetBenchmarkClass, targetProcess=targetProcess, targetBenchmarkClass=targetBenchmarkClass, csvDirPath=csvDirPathForTest)
    # 最適なモデルが線形モデルであることを確認
    assert linSeries["objectBestModelName"] == "ModelLin"
    # 説明変数に対するMAPEが最小のモデルを用いて予測対象の関数コール回数を予測した時の絶対相対誤差率が非常に小さいことを確認

    # 対数モデルが最適となる場合
    # 目的変数を対数モデルが最適となるように設定する
    responseVariableY = 2 * np.log10(explanatoryVariableX) + 3
    logSeries = returnSeriesOfData(benchmarkName=benchmarkName, functionName=f"{functionName}", rawX=explanatoryVariableX, rawY=responseVariableY,
                                   fixProcessOrClass=fixProcessOrClass, fixed=targetBenchmarkClass, targetProcess=targetProcess, targetBenchmarkClass=targetBenchmarkClass, csvDirPath=csvDirPathForTest)
    # 最適なモデルが対数モデルであることを確認
    assert logSeries["objectBestModelName"] == "ModelLog"
    # 説明変数に対するMAPEが最小のモデルを用いて予測対象の関数コール回数を予測した時の絶対相対誤差率が非常に小さいことを確認

In [ ]:
@pytest.fixture()
def test_generateAllBranchFunctionCSVData():
    benchmarkName = "branch"
    fileNamePrefix = f"pprof_{benchmarkName}"
    classes = ["A", "B", "C", "D"]
    explanatoryVariableX = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    responseVariableY = [48, 52, 60, 76, 108, 172, 300, 300, 300]
    functionNames = []
    for i in range(4):
        functionNames.append(f"{benchmarkName}_0{i}")
    for benchmarkClass in classes:
        for process in explanatoryVariableX:
            fileName = f"{fileNamePrefix}{benchmarkClass}{process}.csv"
            filePath = f"/tmp/{fileName}"
            with open(filePath, 'w') as f:
                f.write("Name,#Call\n")
                for functionName in functionNames:
                    functionCallNum = responseVariableY[explanatoryVariableX.index(process)]
                    f.write(f"{functionName},{functionCallNum}")
